#**Cláusula JOIN.**

Hasta ahora hemos visto consultas SQL que extraen datos desde una tabla a la vez. Tomando, por ejemplo, la base de datos Chinook y las múltiples consultas (con operadores condicionales, agrupación y agregación). Sin embargo, lo central de una BD relacional es la relación que existen entre sus tablas. Mirando el esquema de la BD Chinook constatamos lo siguiente:

- Un álbum puede tener muchas canciones/ una canción pertenece a un álbum.
- Un género musical puede tener muchas canciones / una canción pertenece a un Género Musical.
- Muchas canciones pueden pertenecer a muchas playlist.
- Una canción puede tener múltiples medios.
- Múltiples canciones pueden existir en Múltiples Facturas.

<center><img src="https://drive.google.com/uc?id=1fKhtlmyFD2Vryk3ubFX4g6FsS0PNLNaZ" alt="drawing" width=800px/></center>

Revisemos de forma manual las relaciones existentes entre las tablas `Track` y `Album`:


In [ ]:
# prompt: dame el código para conectar con drive

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import sqlite3
import pandas as pd
#Generamos la conexion
conn = sqlite3.connect("/content/drive/MyDrive/SQL/Chinook_Sqlite.sqlite")
cursor = conn.cursor()

In [ ]:
#Imprimimos el catálogo de tablas solo para hacer Check de que todo cargó correctamente
pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';",conn)

,name
0,Album
1,Artist
2,Customer
3,Employee
4,Genre
5,Invoice
6,InvoiceLine
7,MediaType
8,Playlist
9,PlaylistTrack


In [ ]:
pd.read_sql("PRAGMA table_info('Track')",conn)

,cid,name,type,notnull,dflt_value,pk
0,0,TrackId,INTEGER,1,None,1
1,1,Name,NVARCHAR(200),1,None,0
2,2,AlbumId,INTEGER,0,None,0
3,3,MediaTypeId,INTEGER,1,None,0
4,4,GenreId,INTEGER,0,None,0
5,5,Composer,NVARCHAR(220),0,None,0
6,6,Milliseconds,INTEGER,1,None,0
7,7,Bytes,INTEGER,0,None,0
8,8,UnitPrice,"NUMERIC(10,2)",1,None,0


In [ ]:
# Desde la tabla Track obtenemos las columnas TrackId, el Nombre, y el identificador de album (AlbumId). Imprimiendo los primeros 5 elementros de la consulta
pd.read_sql("SELECT TrackId, Name, AlbumId FROM Track;", conn).head()

,TrackId,Name,AlbumId
0,1,For Those About To Rock (We Salute You),1
1,2,Balls to the Wall,2
2,3,Fast As a Shark,3
3,4,Restless and Wild,3
4,5,Princess of the Dawn,3


In [ ]:
# Desde la tabla Album obtenemos el identificador del album (AlbumId), el nombre del album y el Id del artista.
pd.read_sql("SELECT * FROM Album;", conn).head()

,AlbumId,Title,ArtistId
0,1,For Those About To Rock We Salute You,1
1,2,Balls to the Wall,2
2,3,Restless and Wild,2
3,4,Let There Be Rock,1
4,5,Big Ones,3


Ambas tablas tienen en común el atributo **AlbumId**, podemos usar ese atributo para saber el nombre del álbum al cual pertenece cada canción.

Manualmente, nosotros podemos hacer el proceso de ir imprimiendo por pantalla haciendo dos select (por ej: obtener el AlbumId de la tabla Album en un ciclo for, y luego hacer una consulta en la tabla Track donde AlbumId = Album.AlbumId). El problema de hacer esto, es que es un proceso ineficiente al hacer múltiples consultas consecutivas. Para evitar esto debemos utilizar las cláusulas `JOIN` con el fin de hacer este cálculo usando SQL. Específicamente vamos a ver la primera forma de hacerlo, utilizando la cláusula **INNER JOIN**

# INNER JOIN

El `INNER JOIN` va a generar una tabla lógica combinando (concatenando) filas, desde diferentes tablas, que tienen el mismo valor en un atributo que tienen en común. Para hacer el pareo entre filas, necesitamos especificar el campo en común que existe entre las tablas. Con esto, el `INNER JOIN` fuerza a que valores de una tabla A que no se encuentren en una tabla B, sean excluidos. Un diagrama de esta interacción se presenta a continuación:

<center><img src="https://drive.google.com/uc?id=1EIU8JgCgKJx1deGzbjthfnfBU49S8Apv"></center>

La sintaxis para definir un `INNER JOIN` en SQL corresponde a la siguiente:


```SQL
SELECT * FROM Tabla_A INNER JOIN Tabla_B ON Tabla_A.id = Tabla_B.id_a;
SELECT * FROM Tabla_A ta INNER JOIN Tabla_B tb ON ta.id = tb.id_a;
SELECT * FROM Tabla_A ta INNER JOIN Tabla_B tb USING(id); -- Esta cláusula funciona siempre y cuando el campo tiene el mismo nombre en ambas tablas.
```
Realicemos el ejercicio de agregar el nombre del disco para cada canción en la Tabla Track, utilizando la Tabla Albums:

Si miramos la tabla `Track` nos damos cuenta que el nombre del álbum al cual pertenece una canción no se encuentra de manera explícita, sólo tiene un número `AlbumId`. Sin embargo, podemos usar el atributo `AlbumId` en un JOIN entre las tablas `Track` y `Album`.   

In [ ]:
sqlQuery = """
            SELECT t.TrackId, t.Name, a.Title as AlbumTitle, t.UnitPrice
            FROM Track t
            INNER JOIN Album a
            ON t.AlbumId = a.AlbumId;
          """
pd.read_sql(sqlQuery, conn)

,TrackId,Name,AlbumTitle,UnitPrice
0,1,For Those About To Rock (We Salute You),For Those About To Rock We Salute You,0.495
1,2,Balls to the Wall,Balls to the Wall,0.990
2,3,Fast As a Shark,Restless and Wild,0.990
3,4,Restless and Wild,Restless and Wild,0.990
4,5,Princess of the Dawn,Restless and Wild,0.990
...,...,...,...,...
3498,3499,Pini Di Roma (Pinien Von Rom) \ I Pini Della V...,Respighi:Pines of Rome,0.990
3499,3500,"String Quartet No. 12 in C Minor, D. 703 ""Quar...",Schubert: The Late String Quartets & String Qu...,0.990
3500,3501,"L'orfeo, Act 3, Sinfonia (Orchestra)",Monteverdi: L'Orfeo,0.990
3501,3502,"Quintet for Horn, Violin, 2 Violas, and Cello ...",Mozart: Chamber Music,0.990


Dado que en ambas tablas el atributo utilizado para el JOIN tiene el mismo nombre podemos usar `USING` de la sgte manera:

In [ ]:
sqlQuery = """
    SELECT t.TrackId, t.Name, a.Title as AlbumTitle, t.UnitPrice
    FROM Track t
    INNER JOIN Album a
    USING(AlbumId);
"""
pd.read_sql(sqlQuery, conn).head()

,TrackId,Name,AlbumTitle,UnitPrice
0,1,For Those About To Rock (We Salute You),For Those About To Rock We Salute You,0.495
1,2,Balls to the Wall,Balls to the Wall,0.990
2,3,Fast As a Shark,Restless and Wild,0.990
3,4,Restless and Wild,Restless and Wild,0.990
4,5,Princess of the Dawn,Restless and Wild,0.990


#**Hacer Join con múltiples tablas**

SQL otorga flexibilidad para poder hacer `JOIN` con múltiples tablas, la idea siempre es hacer un match en la relación existente entre las múltiples tablas. Una consulta de ejemplo puede ser la siguiente:

```SQL
SELECT *
FROM Tabla_A ta
INNER JOIN Tabla_B tb
ON ta.id = tb.id_a
INNER JOIN Tabla_C tc
ON tb.id = tc.id_b
```

En este caso, tenemos tres tablas que tienen relaciones lógicas entre sí, y por ende, dos clausulas `INNER JOIN` y sub-claúsulas `ON`. <br>
Cómo respondemos a la siguiente consulta:<br>
**Para cada canción almacenada en la BD, cuál es el nombre, su estilo musical y el nombre del álbum al cual pertnenece?**<br>
 Si miramos el esquema de la BD vemos que necesitamos datos desde 3 tablas (`Track`, `Album` y `Genre`) y la relación entre ellas es la sgte: <br>
<center>
$Album ← Track → Genre$
</center>
y la consulta SQL es la siguiente:

In [ ]:
sqlQuery = """
    SELECT t.TrackId, t.Name as "Track Name", a.Title as "Album Title", gr.name as Genre
    FROM Track t
    INNER JOIN Album a
    ON t.AlbumId = a.AlbumId
    INNER JOIN Genre gr
    on t.GenreId = gr.GenreId
    ;
"""
pd.read_sql(sqlQuery, conn)

,TrackId,Track Name,Album Title,Genre
0,1,For Those About To Rock (We Salute You),For Those About To Rock We Salute You,Rock
1,2,Balls to the Wall,Balls to the Wall,Rock
2,3,Fast As a Shark,Restless and Wild,Rock
3,4,Restless and Wild,Restless and Wild,Rock
4,5,Princess of the Dawn,Restless and Wild,Rock
...,...,...,...,...
3498,3499,Pini Di Roma (Pinien Von Rom) \ I Pini Della V...,Respighi:Pines of Rome,Classical
3499,3500,"String Quartet No. 12 in C Minor, D. 703 ""Quar...",Schubert: The Late String Quartets & String Qu...,Classical
3500,3501,"L'orfeo, Act 3, Sinfonia (Orchestra)",Monteverdi: L'Orfeo,Classical
3501,3502,"Quintet for Horn, Violin, 2 Violas, and Cello ...",Mozart: Chamber Music,Classical


Si ahora necesitamos:<br>
**Para cada canción almacenada en la BD, cuál es su nombre, el nombre del álbum al cual pertnenece y el artista que la ejecuta?**<br>

la consulta SQL es la siguiente:

In [ ]:
sqlQuery = """
    SELECT at.name as Artist, t.Name as "Track Name", a.Title as "Album Title"
    FROM Track t
    INNER JOIN Album a
    ON t.AlbumId = a.AlbumId
    INNER JOIN artist at
    ON a.ArtistId = at.ArtistId;
"""
pd.read_sql(sqlQuery, conn).head(20)

,Artist,Track Name,Album Title
0,AC/DC,For Those About To Rock (We Salute You),For Those About To Rock We Salute You
1,Accept,Balls to the Wall,Balls to the Wall
2,Accept,Fast As a Shark,Restless and Wild
3,Accept,Restless and Wild,Restless and Wild
4,Accept,Princess of the Dawn,Restless and Wild
5,AC/DC,Put The Finger On You,For Those About To Rock We Salute You
6,AC/DC,Let's Get It Up,For Those About To Rock We Salute You
7,AC/DC,Inject The Venom,For Those About To Rock We Salute You
8,AC/DC,Snowballed,For Those About To Rock We Salute You
9,AC/DC,Evil Walks,For Those About To Rock We Salute You


Cabe destacar que todos los operadores condicionales, agrupamientos y agregaciones se pueden seguir aplicando:

In [ ]:
# Aumentando la consulta anterior, se incluye el precio unitario siempre y cuando su valor sea < 1.5
sqlQuery = """
    SELECT at.name as Artist, t.Name as "Track Name", a.Title as "Album Title", t.UnitPrice as "Unit Price"
    FROM Track t
    INNER JOIN Album a
    ON t.AlbumId = a.AlbumId
    INNER JOIN artist at
    ON a.ArtistId = at.ArtistId
    WHERE t.Unitprice > 1.5;
"""
pd.read_sql(sqlQuery, conn).head(20)

,Artist,Track Name,Album Title,Unit Price


Otro ejemplo, artistas que tienen más de 20 canciones junto con la duración promedios de estas.

In [ ]:
sqlQuery = """
    SELECT at.name as Artist, COUNT() as "Number of Tracks", AVG(t.Milliseconds)/(60*1000) as "Average duration"

    FROM Track t
    INNER JOIN Album a
    ON t.AlbumId = a.AlbumId

    INNER JOIN artist at
    ON a.ArtistId = at.ArtistId

    GROUP BY at.name
    HAVING "Number of Tracks" > 20
    ORDER BY Artist
    ;
"""
pd.read_sql(sqlQuery, conn).head(20)

,Artist,Number of Tracks,Average duration
0,Amy Winehouse,23,4.043375
1,Antônio Carlos Jobim,31,3.832465
2,Audioslave,40,4.439828
3,Battlestar Galactica (Classic),24,48.759572
4,Caetano Veloso,21,3.792931
5,Chico Buarque,34,3.860609
6,Chico Science & Nação Zumbi,36,3.359001
7,Cidade Negra,31,3.965759
8,Creedence Clearwater Revival,40,3.577712
9,Cássia Eller,30,3.821722


#**Natural Join**

Varios motores de base de datos implementan una forma sencilla de implementar los `INNER JOIN`. En este caso, la cláusula `NATURAL JOIN` implementa una búsqueda automática de los atributos a los cuales existe la relación para poder hacer la tabla lógica. En este caso la sintaxis para poder aplicar dicha cláusula corresponde a:

```SQL
SELECT * FROM Tabla_A ta NATURAL JOIN Tabla_B tb;
```

Veámoslo con un ejemplo, transformando un `INNER JOIN` a `NATURAL JOIN`:


In [ ]:
sqlQuery = """
    SELECT t.TrackId, t.Name, a.Title as AlbumTitle, t.UnitPrice
    FROM Track t
    NATURAL JOIN Album a;
"""
pd.read_sql(sqlQuery, conn).head()

,TrackId,Name,AlbumTitle,UnitPrice
0,1,For Those About To Rock (We Salute You),For Those About To Rock We Salute You,0.495
1,2,Balls to the Wall,Balls to the Wall,0.990
2,3,Fast As a Shark,Restless and Wild,0.990
3,4,Restless and Wild,Restless and Wild,0.990
4,5,Princess of the Dawn,Restless and Wild,0.990


#**Self-Joins**

Hasta ahora hemos visto cómo hacer operaciones `JOIN` entre múltiples tablas. Aunque sea poco intuitivo, podemos hacer un join sobre una misma tabla con el fin de explotar relaciones existentes entre campos de una misma tabla. Un diagrama de la operación es el siguiente:

<center><img src="https://drive.google.com/uc?id=1mK9ZugQwymZq5pLhpMppRvSK21BvDfW0"></center>

Para entender que está pasando acá, veamos de forma práctica utilizando la tabla de empleados (Employee):
<center><img src="https://drive.google.com/uc?id=10wjBeggtq8GvHoPfhAE1bBPDf9tVCLx9"></center>

En este caso, podemos ver que existe una relación donde cada empleado, le reporta a otro empleado (su superior y así sucesivamente). En este caso, puede hacer sentido la utilización de un **SELF JOIN**. En términos de sintaxis es similar a los joins vistos anteriormente, solo hay que tener en cuenta los alias de las tablas.

In [ ]:
sqlQuery = """
    SELECT e.FirstName, e.LastName, e.Title, (e_reports.Firstname || ' ' || e_reports.Lastname) as nombre_Superior, e_reports.Title as cargo_superior
    FROM Employee e

    INNER JOIN Employee e_reports
    ON e.ReportsTo = e_reports.EmployeeId
    ;
"""
pd.read_sql(sqlQuery, conn).head(10)

,FirstName,LastName,Title,nombre_Superior,cargo_superior
0,Nancy,Edwards,Sales Manager,Andrew Adams,General Manager
1,Jane,Peacock,Sales Support Agent,Nancy Edwards,Sales Manager
2,Margaret,Park,Sales Support Agent,Nancy Edwards,Sales Manager
3,Steve,Johnson,Sales Support Agent,Nancy Edwards,Sales Manager
4,Michael,Mitchell,IT Manager,Andrew Adams,General Manager
5,Robert,King,IT Staff,Michael Mitchell,IT Manager
6,Laura,Callahan,IT Staff,Michael Mitchell,IT Manager


Veamos cuantos elementos retorna la consulta con join y la consulta normal en la tabla de empleados **¿Que está sucediendo?**

In [ ]:
sqlQuery = """
    SELECT COUNT()
    FROM Employee e

    INNER JOIN Employee e_reports
    ON e.ReportsTo = e_reports.EmployeeId
    ;
"""
print("Consulta con Join ====")
print(pd.read_sql(sqlQuery, conn))


sqlQuery = """
    SELECT COUNT()
    FROM Employee e
    ;
"""
print("Consulta sin Join ====")
print(pd.read_sql(sqlQuery, conn))

Consulta con Join ====
   COUNT()
0        7
Consulta sin Join ====
   COUNT()
0        8


In [ ]:
pd.read_sql("SELECT FirstName, LastName, Title, ReportsTo FROM Employee",conn)

,FirstName,LastName,Title,ReportsTo
0,Andrew,Adams,General Manager,NaN
1,Nancy,Edwards,Sales Manager,1.0
2,Jane,Peacock,Sales Support Agent,2.0
3,Margaret,Park,Sales Support Agent,2.0
4,Steve,Johnson,Sales Support Agent,2.0
5,Michael,Mitchell,IT Manager,1.0
6,Robert,King,IT Staff,6.0
7,Laura,Callahan,IT Staff,6.0


Andrew Adams es el gerente general y no tiene un superior en la empresa.

#**Actividad 1**
Con la BD Baseball `baseball.sqlite`:
- Obtenga el nombre (columnas name_first y name_last concatenadas), fecha de nacimiento (ver las columnas que empiecen con birth), ciudad de nacimiento, peso, altura y salario para los jugadores (Ver tabla Player y Salary)
- Modifique la consulta anterior para hacer agrupamiento por jugador, y muestre el salario promedio para cada uno de los jugadores, ordene por dicho valor de forma descendente
- La tabla appearences hay una columna que se llama g_all, esta hace referencia a la cantidad de juegos que a participado el jugador, Obtenga el nombre y la suma acumulada de todos los juegos para cada jugador
- Modifique la consulta anterior filtrando por todos los jugadores que ya hayan fallecido, y que tengan mas de 2500 juegos acumulados. Añada la fecha de fallecimiento para cada uno de los jugadores como atributo adicional. **Como pista, un jugador ha fallecido si alguna de las columnas asociadas a death es distinta de un string vacío ('')**
- Repita la consulta anterior para la consulta que están vivos, pero obteniendo la cantidad de juegos que han estado en primera base, segunda base y tercera base (g_1b, g2_b y g3_b)


In [ ]:
#Cargue los datos
#Su código aqui...
conn2 = sqlite3.connect("/content/drive/MyDrive/SQL/baseball.sqlite")
cursor2 = conn2.cursor()

In [ ]:
pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';",conn2)

,name
0,all_star
1,appearances
2,manager_award
3,player_award
4,manager_award_vote
5,player_award_vote
6,batting
7,batting_postseason
8,player_college
9,fielding


##**Consulta 1**

In [ ]:
#veo la tabla player
pd.read_sql("PRAGMA table_info('Player')",conn2)

,cid,name,type,notnull,dflt_value,pk
0,0,player_id,TEXT,0,None,0
1,1,birth_year,NUMERIC,0,None,0
2,2,birth_month,NUMERIC,0,None,0
3,3,birth_day,NUMERIC,0,None,0
4,4,birth_country,TEXT,0,None,0
5,5,birth_state,TEXT,0,None,0
6,6,birth_city,TEXT,0,None,0
7,7,death_year,NUMERIC,0,None,0
8,8,death_month,NUMERIC,0,None,0
9,9,death_day,NUMERIC,0,None,0


In [ ]:
#veo la tabla salary
pd.read_sql("PRAGMA table_info('Salary')",conn2)

,cid,name,type,notnull,dflt_value,pk
0,0,year,INTEGER,0,None,0
1,1,team_id,TEXT,0,None,0
2,2,league_id,TEXT,0,None,0
3,3,player_id,TEXT,0,None,0
4,4,salary,INTEGER,0,None,0


In [ ]:
#Obtenga el nombre (columnas name_first y name_last concatenadas), fecha de nacimiento (ver las columnas que empiecen con birth), ciudad de nacimiento, peso, altura y salario para los jugadores (Ver tabla Player y Salary)
#Su código aqui...
# Desde la tabla Track obtenemos las columnas TrackId, el Nombre, y el identificador de album (AlbumId). Imprimiendo los primeros 5 elementros de la consulta
pd.read_sql("SELECT (name_first || ' ' || name_last) as name_fl,birth_year, birth_month, birth_day, birth_city, weight, height FROM Player;", conn2).head()

,name_fl,birth_year,birth_month,birth_day,birth_city,weight,height
0,David Aardsma,1981,12,27,Denver,220,75
1,Hank Aaron,1934,2,5,Mobile,180,72
2,Tommie Aaron,1939,8,5,Mobile,190,75
3,Don Aase,1954,9,8,Orange,190,75
4,Andy Abad,1972,8,25,Palm Beach,184,73


In [ ]:
pd.read_sql("SELECT salary FROM Salary;", conn2).head()

,salary
0,870000
1,550000
2,545000
3,633333
4,625000


##**Consulta 2**

In [ ]:
# Modifique la consulta anterior para hacer agrupamiento por jugador, y muestre el salario promedio para cada uno de los jugadores, ordene por dicho valor de forma descendente
#Su código aqui...
pd.read_sql("""
    SELECT Player.player_id,
           (Player.name_first || ' ' || Player.name_last) AS name_fl,
           ROUND(AVG(Salary.salary)) AS avg_salary
    FROM Player
    LEFT JOIN Salary ON Player.player_id = Salary.player_id
    GROUP BY Player.player_id
    ORDER BY avg_salary DESC;
""", conn2)


,player_id,name_fl,avg_salary
0,tanakma01,Masahiro Tanaka,22000000.0
1,rodrial01,Alex Rodriguez,17972202.0
2,howarry01,Ryan Howard,15525500.0
3,teixema01,Mark Teixeira,14703846.0
4,jeterde01,Derek Jeter,13927268.0
...,...,...,...
18841,abbeybe01,Bert Abbey,NaN
18842,abbated01,Ed Abbaticchio,NaN
18843,abadijo01,John Abadie,NaN
18844,aaronto01,Tommie Aaron,NaN


##**Consulta 3**

In [ ]:
# La tabla appearences hay una columna que se llama g_all, esta hace referencia a la cantidad de juegos que a participado el jugador, Obtenga el nombre y la suma acumulada de todos los juegos para cada jugador
#Su código aqui...
#veo la tabla appearances
pd.read_sql("PRAGMA table_info('appearances')",conn2)

,cid,name,type,notnull,dflt_value,pk
0,0,year,INTEGER,0,None,0
1,1,team_id,TEXT,0,None,0
2,2,league_id,TEXT,0,None,0
3,3,player_id,TEXT,0,None,0
4,4,g_all,NUMERIC,0,None,0
5,5,gs,NUMERIC,0,None,0
6,6,g_batting,INTEGER,0,None,0
7,7,g_defense,NUMERIC,0,None,0
8,8,g_p,INTEGER,0,None,0
9,9,g_c,INTEGER,0,None,0


In [ ]:
pd.read_sql("""
    SELECT Player.player_id,
           (Player.name_first || ' ' || Player.name_last) AS name,
           SUM(Appearances.g_all) AS total_games
    FROM Player
    LEFT JOIN Appearances ON Player.player_id = Appearances.player_id
    GROUP BY Player.player_id
    ORDER BY total_games DESC;
""", conn2)

,player_id,name,total_games
0,rosepe01,Pete Rose,3562.0
1,yastrca01,Carl Yastrzemski,3308.0
2,aaronha01,Hank Aaron,3298.0
3,henderi01,Rickey Henderson,3081.0
4,cobbty01,Ty Cobb,3034.0
...,...,...,...
18841,bancrfr99,Frank Bancroft,NaN
18842,arrueer01,Erisbel Arruebarrena,NaN
18843,armoubi99,Bill Armour,NaN
18844,adairbi99,Bill Adair,NaN


##**Consulta 4**

In [ ]:
# Modifique la consulta anterior filtrando por todos los jugadores que ya hayan fallecido, y que tengan mas de 2500 juegos acumulados. Añada la fecha de fallecimiento para cada uno de los jugadores como atributo adicional. **Como pista, un jugador ha fallecido si alguna de las columnas asociadas a death es distinta de un string vacio ('')**
#Su código aqui...
pd.read_sql("""
    SELECT Player.player_id,
           (Player.name_first || ' ' || Player.name_last) AS name,
           SUM(Appearances.g_all) AS total_games,
           Player.death_year || '-' ||
           Player.death_month || '-' ||
           Player.death_day AS death_date
    FROM Player
    LEFT JOIN Appearances ON Player.player_id = Appearances.player_id
    WHERE Player.death_year != ''
          AND Player.death_month != ''
          AND Player.death_day != ''
    GROUP BY Player.player_id
    HAVING total_games > 2500
    ORDER BY total_games DESC;
""", conn2)


,player_id,name,total_games,death_date
0,cobbty01,Ty Cobb,3034,1961-7-17
1,musiast01,Stan Musial,3026,2013-1-19
2,collied01,Eddie Collins,2826,1951-3-25
3,wagneho01,Honus Wagner,2794,1955-12-6
4,speaktr01,Tris Speaker,2789,1958-12-8
5,ottme01,Mel Ott,2730,1958-11-21
6,maranra01,Rabbit Maranville,2670,1954-1-5
7,wanerpa01,Paul Waner,2549,1965-8-29
8,bankser01,Ernie Banks,2528,2015-1-23
9,ansonca01,Cap Anson,2524,1922-4-14


##**Consulta 5**

In [ ]:
#Repita la consulta anterior para la consulta que estan vivos, pero obteniendo la cantidad de juegos que han estado en primera base, segunda base y tercera base (g_1b, g2_b y g3_b)
#Su código aqui...
pd.read_sql("""
    SELECT Player.player_id,
           (Player.name_first || ' ' || Player.name_last) AS name,
           SUM(Appearances.g_1b) AS games_1b,
           SUM(Appearances.g_2b) AS games_2b,
           SUM(Appearances.g_3b) AS games_3b
    FROM Player
    LEFT JOIN Appearances ON Player.player_id = Appearances.player_id
    WHERE (Player.death_year = '' OR Player.death_year IS NULL)
    GROUP BY Player.player_id
    ORDER BY (games_1b + games_2b + games_3b) DESC;
""", conn2)


,player_id,name,games_1b,games_2b,games_3b
0,robinbr01,Brooks Robinson,0.0,25.0,2870.0
1,perezto01,Tony Perez,1780.0,1.0,760.0
2,morgajo02,Joe Morgan,0.0,2527.0,3.0
3,beltrad01,Adrian Beltre,0.0,1.0,2483.0
4,nettlgr01,Graig Nettles,14.0,0.0,2413.0
...,...,...,...,...,...
9505,bristda99,Dave Bristol,NaN,NaN,NaN
9506,bolesjo99,John Boles,NaN,NaN,NaN
9507,bevinte99,Terry Bevington,NaN,NaN,NaN
9508,arrueer01,Erisbel Arruebarrena,NaN,NaN,NaN


#**Fin Actividad 1**

# <font color='purple'> __EXPERIMENTO__: </font>
 Calcularemos el salario promedio por jugador por team, y agregaremos el nombre del team

Para revisar este experimento primero necesitamos conocer las columnas de la tabla team, para ver si tenemos la variable identificadora de team y otra con el nombre

In [ ]:
#veo la tabla team
pd.read_sql("PRAGMA table_info('team')",conn2)

,cid,name,type,notnull,dflt_value,pk
0,0,year,INTEGER,0,None,0
1,1,league_id,TEXT,0,None,0
2,2,team_id,TEXT,0,None,0
3,3,franchise_id,TEXT,0,None,0
4,4,div_id,TEXT,0,None,0
5,5,rank,INTEGER,0,None,0
6,6,g,INTEGER,0,None,0
7,7,ghome,NUMERIC,0,None,0
8,8,w,INTEGER,0,None,0
9,9,l,INTEGER,0,None,0


In [ ]:
#encontrar el promedio de salario para team específicos.

query_teams = """
SELECT t.team_id, t.name, AVG(s.salary) AS average_salary
FROM Salary s
JOIN Team t ON s.team_id = t.team_id
GROUP BY t.team_id, t.name;
"""

df_team_salaries = pd.read_sql(query_teams, conn2)
print(df_team_salaries)


   team_id                           name  average_salary
0      ANA                 Anaheim Angels    1.895109e+06
1      ARI           Arizona Diamondbacks    2.479160e+06
2      ATL                 Atlanta Braves    2.206239e+06
3      BAL              Baltimore Orioles    1.915746e+06
4      BOS               Boston Americans    2.856617e+06
5      BOS                 Boston Red Sox    2.856617e+06
6      CAL              California Angels    7.390732e+05
7      CHA              Chicago White Sox    2.099929e+06
8      CHN                  Chicago Colts    2.255380e+06
9      CHN                   Chicago Cubs    2.255380e+06
10     CHN                Chicago Orphans    2.255380e+06
11     CHN        Chicago White Stockings    2.255380e+06
12     CIN             Cincinnati Redlegs    1.717922e+06
13     CIN                Cincinnati Reds    1.717922e+06
14     CLE                Cleveland Blues    1.618959e+06
15     CLE             Cleveland Bronchos    1.618959e+06
16     CLE    

## <font color='purple'>Fin experimento </font>

#**Left - Right - Outer Joins**

En todos los ejemplos que hemos visto hasta ahora, hacen uso del `INNER JOIN`, y tal como vimos en el ejemplo previo a la dinámica, esta cláusula tiene algunas desventajas. Si bien puede hacer el match según la relación que nosotros definamos, vamos a dejar a fuera todos los registros que no cumplan la condición perdiendo información en el proceso. Como tal, nosotros no sabíamos quien era `Andrew Adams` (o a quien le reportaba). Para esto existen otras clausulas las cuales provienen de la cláusula `OUTER JOIN` y las cuales permiten extraer información de las tablas a las cuales estamos uniendo, sin perder los resultados cuando no existe la relación lógica entre tablas.

##**Left Outer Join:**

La cláusula `LEFT OUTER JOIN`, trae todos los registros que cumplan la condición de la relación entre las tablas **y todos los registros de la tabla que está a la izquierda del JOIN**. De forma Gráfica tenemos la siguiente imagen:

<center><img src="https://drive.google.com/uc?id=111-3318II31xb4GwGre5ls6Vh9Ff7ODg"></center>

y la sintaxis de dicha cláusula es bastante similar a las cláusulas de `Inner Join`, reemplazando solo la cláusula por `Left Outer Join`:

```SQL
SELECT * FROM Tabla_a ta LEFT OUTER JOIN Tabla_b tb on ta.id = tb.id_a
```
En este caso, si para un registro de ta no existe la condicion `ta.id = tb.id_a` de todas formas se retornará el registro de `ta`, pero para las columnas de `tb` retornará valores nulos. Podemos verlo como ejemplo en la consulta de los empleados, donde en vez de el `ÌNNER JOIN` vamos a utilizar un `LEFT OUTER JOIN`


In [ ]:
sqlQuery = """
    SELECT e.FirstName, e.LastName, e.Title, (e_reports.Firstname || ' ' || e_reports.Lastname) as nombre_Superior, e_reports.Title as cargo_superior
    FROM Employee e

    LEFT OUTER JOIN Employee e_reports
    ON e.ReportsTo = e_reports.EmployeeId
    ;
"""
pd.read_sql(sqlQuery, conn).head(10)

,FirstName,LastName,Title,nombre_Superior,cargo_superior
0,Andrew,Adams,General Manager,None,None
1,Nancy,Edwards,Sales Manager,Andrew Adams,General Manager
2,Jane,Peacock,Sales Support Agent,Nancy Edwards,Sales Manager
3,Margaret,Park,Sales Support Agent,Nancy Edwards,Sales Manager
4,Steve,Johnson,Sales Support Agent,Nancy Edwards,Sales Manager
5,Michael,Mitchell,IT Manager,Andrew Adams,General Manager
6,Robert,King,IT Staff,Michael Mitchell,IT Manager
7,Laura,Callahan,IT Staff,Michael Mitchell,IT Manager


y viendo la cantidad de registros que retorna, coincide con la cantidad de elementos que hay en la tabla de empleados.

In [ ]:
sqlQuery = """
    SELECT COUNT()
    FROM Employee e

    LEFT OUTER JOIN Employee e_reports
    ON e.ReportsTo = e_reports.EmployeeId
    ;
"""
print("Consulta con Join ====")
print(pd.read_sql(sqlQuery, conn))

Consulta con Join ====
   COUNT()
0        8


##**Right Outer Join**

La cláusula `RIGHT OUTER JOIN`, trae todos los registros que cumplan la condición de la relación entre las tablas **y todos los registros de la tabla que está a la Derecha del JOIN**. De forma Gráfica tenemos la siguiente imagen:


<center><img src="https://drive.google.com/uc?id=19qpyL9G1yh4CwDvBJTJpOQvT619shJDi"></center>

y la sintaxis de dicha cláusula es similar a las cláusulas de `LEFT OUTER JOIN`, reemplazando solo `LEFT` por `RIGHT` en la cláusula:

```SQL
SELECT * FROM Tabla_a ta RIGHT OUTER JOIN Tabla_b tb on ta.id = tb.id_a
```
En este caso, si para un registro de ta no existe la condición `tb.id_a = ta.id` de todas formas se retornará el registro de `tb`, pero para las columnas de `ta` retornara valores nulos.

Ahora, existe un **problema en SQLite**... Veamos que sucede si aplicamos un `RIGHT OUTER JOIN` al mismo ejemplo de la tabla de empleados:


In [ ]:
sqlQuery = """
    SELECT e.FirstName, e.LastName, e.Title, (e_reports.Firstname || ' ' || e_reports.Lastname) as nombre_Superior, e_reports.Title as cargo_superior
    FROM Employee e

    RIGHT OUTER JOIN Employee e_reports
    ON e.ReportsTo = e_reports.EmployeeId
    ;
"""
pd.read_sql(sqlQuery, conn).head(10)

DatabaseError: Execution failed on sql '
    SELECT e.FirstName, e.LastName, e.Title, (e_reports.Firstname || ' ' || e_reports.Lastname) as nombre_Superior, e_reports.Title as cargo_superior
    FROM Employee e

    RIGHT OUTER JOIN Employee e_reports
    ON e.ReportsTo = e_reports.EmployeeId
    ;
': RIGHT and FULL OUTER JOINs are not currently supported

<img src="https://www.dictionary.com/e/wp-content/uploads/2018/03/Thinking_Face_Emoji-Emoji-Island-300x300.png">

La pregunta entonces: **¿Que podemos hacer para tratar de solucionar este problema de implementación?** la respuesta está en el siguiente código:


In [ ]:
sqlQuery = """
    SELECT e.FirstName, e.LastName, e.Title, (e_reports.Firstname || ' ' || e_reports.Lastname) as nombre_Superior, e_reports.Title as cargo_superior
    FROM Employee e_reports

    LEFT OUTER JOIN Employee e
    ON e.ReportsTo = e_reports.EmployeeId
    ;
"""
pd.read_sql(sqlQuery, conn)

,FirstName,LastName,Title,nombre_Superior,cargo_superior
0,Nancy,Edwards,Sales Manager,Andrew Adams,General Manager
1,Michael,Mitchell,IT Manager,Andrew Adams,General Manager
2,Jane,Peacock,Sales Support Agent,Nancy Edwards,Sales Manager
3,Margaret,Park,Sales Support Agent,Nancy Edwards,Sales Manager
4,Steve,Johnson,Sales Support Agent,Nancy Edwards,Sales Manager
5,None,None,None,Jane Peacock,Sales Support Agent
6,None,None,None,Margaret Park,Sales Support Agent
7,None,None,None,Steve Johnson,Sales Support Agent
8,Robert,King,IT Staff,Michael Mitchell,IT Manager
9,Laura,Callahan,IT Staff,Michael Mitchell,IT Manager


Haciendo ese traspaso, nosotros podemos emular la funcionalidad que existe para la cláusula `RIGHT OUTER JOIN`. Solo destacar que esto es un problema que existe en SQLite, tal como detallan en el siguiente documento: [SQL Omitted Features](https://sqlite.org/omitted.html)

##**Full Outer Join**

La clausula `FULL OUTER JOIN`, corresponde a la union de `LEFT OUTER JOIN` y de `RIGHT OUTER JOIN`. Esto quiere decir que va a incluir los registros de ambas tablas al momento de hacer el proceso de emparejamiento según las relaciones lógicas



<center><img src="https://drive.google.com/uc?id=1bH5BmaKEMv7yseuaZG8m2C36-LXdAF56"></center>

La sintaxis para esta cláusula es igual para `LEFT OUTER JOIN` y `RIGHT OUTER JOIN`:

```SQL
SELECT * FROM Tabla_a ta FULL OUTER JOIN Tabla_b tb on ta.id = tb.id_a
```

El problema nuevamente ocurre con SQLite no implementando `FULL OUTER JOIN` por lo que tendremos que hacer una solución alterna tal como se hizo para el `RIGHT OUTER JOIN`. Adelantándonos un poco en la materia, tendremos que utilizar un operador que es el operador `UNION` el cual combina múltiples resultados de varias consultas `SELECT`. En este caso para poder emular el `FULL OUTER JOIN`, tenemos que hacer lo siguiente:

In [ ]:
sqlQuery1 = """
    SELECT e.FirstName, e.LastName, e.Title, (e_reports.Firstname || ' ' || e_reports.Lastname) as nombre_Superior, e_reports.Title as cargo_superior
    FROM Employee e

    LEFT OUTER JOIN Employee e_reports
    ON e.ReportsTo = e_reports.EmployeeId

"""

sqlQuery2 = """
    SELECT e.FirstName, e.LastName, e.Title, (e_reports.Firstname || ' ' || e_reports.Lastname) as nombre_Superior, e_reports.Title as cargo_superior
    FROM Employee e_reports

    LEFT OUTER JOIN Employee e
    ON e.ReportsTo = e_reports.EmployeeId

"""

sqlQuery = sqlQuery1 + '    UNION\n' + sqlQuery2
print(sqlQuery)


    SELECT e.FirstName, e.LastName, e.Title, (e_reports.Firstname || ' ' || e_reports.Lastname) as nombre_Superior, e_reports.Title as cargo_superior
    FROM Employee e

    LEFT OUTER JOIN Employee e_reports
    ON e.ReportsTo = e_reports.EmployeeId

    UNION

    SELECT e.FirstName, e.LastName, e.Title, (e_reports.Firstname || ' ' || e_reports.Lastname) as nombre_Superior, e_reports.Title as cargo_superior
    FROM Employee e_reports

    LEFT OUTER JOIN Employee e
    ON e.ReportsTo = e_reports.EmployeeId




In [ ]:
pd.read_sql(sqlQuery, conn)

,FirstName,LastName,Title,nombre_Superior,cargo_superior
0,None,None,None,Jane Peacock,Sales Support Agent
1,None,None,None,Laura Callahan,IT Staff
2,None,None,None,Margaret Park,Sales Support Agent
3,None,None,None,Robert King,IT Staff
4,None,None,None,Steve Johnson,Sales Support Agent
5,Andrew,Adams,General Manager,None,None
6,Jane,Peacock,Sales Support Agent,Nancy Edwards,Sales Manager
7,Laura,Callahan,IT Staff,Michael Mitchell,IT Manager
8,Margaret,Park,Sales Support Agent,Nancy Edwards,Sales Manager
9,Michael,Mitchell,IT Manager,Andrew Adams,General Manager


#**Actividad 2**
Con la BD Baseball `baseball.sqlite`:
- Obtenga el nombre (columnas name_first y name_last concatenadas), fecha de nacimiento (ver las columnas que empiecen con birth), ciudad de nacimiento, peso, altura y salario para los jugadores (Ver tabla Player y Salary). Utilice LEFT OUTER JOIN y RIGHT OUTER JOIN. A simple vista ¿Puede observar valores nulos? ¿Qué sucede en el caso del RIGHT OUTER JOIN?
- Obtenga los nombres, fecha de nacimiento y cantidad de premios que ha obtenido cada jugador. Para esto utilice LEFT OUTER JOIN para obtener incluso a los jugadores que no han recibido algún premio
- Obtenga la cantidad de jugadores que han pasado por una universidad, para esto utilice COUNT(DISTINCT \* ) donde \* corresponde a la columna a contar, para esto vea las tablas College y Player_College. ¿Por qué es necesario ocupar DISTINCT? Genere la consulta sin agrupación y obtenga las columnas que ustedes estimen pertinentes
- Proponga 3 consultas para distintas tablas con el fin de explorar. La restricción es que tiene que haber una cláusula `LEFT OUTER JOIN`, una de `RIGHT OUTER JOIN`, y una de `FULL OUTER JOIN`.


##**Consulta 1**

In [ ]:
#Obtenga el nombre (columnas name_first y name_last concatenadas), fecha de nacimiento (ver las columnas que empiecen con birth), ciudad de nacimiento, peso, altura y salario para los jugadores (Ver tabla Player y Salary). Utilice LEFT OUTER JOIN y RIGHT OUTER JOIN. A simple vista ¿Puede observar valores nulos? ¿Que sucede en el caso del RIGHT OUTER JOIN?
# Su código aquí...
pd.read_sql("""
    SELECT
        p.name_first || ' ' || p.name_last AS name_fl,
        p.birth_year || '-' || p.birth_month || '-' || p.birth_day AS birth_date,
        p.birth_city,
        p.weight,
        p.height,
        s.salary
    FROM Player p
    LEFT OUTER JOIN Salary s ON p.player_id = s.player_id;
""", conn2)

,name_fl,birth_date,birth_city,weight,height,salary
0,David Aardsma,1981-12-27,Denver,220,75,300000.0
1,David Aardsma,1981-12-27,Denver,220,75,387500.0
2,David Aardsma,1981-12-27,Denver,220,75,403250.0
3,David Aardsma,1981-12-27,Denver,220,75,419000.0
4,David Aardsma,1981-12-27,Denver,220,75,500000.0
...,...,...,...,...,...,...
39450,Frank Zupo,1939-8-29,San Francisco,182,71,NaN
39451,Paul Zuvella,1958-10-31,San Mateo,173,72,145000.0
39452,George Zuverink,1924-8-20,Holland,195,76,NaN
39453,Dutch Zwilling,1888-11-2,St. Louis,160,66,NaN


In [ ]:
pd.read_sql("""
    SELECT
        p.name_first || ' ' || p.name_last AS name_fl,
        p.birth_year || '-' || p.birth_month || '-' || p.birth_day AS birth_date,
        p.birth_city,
        p.weight,
        p.height,
        s.salary
    FROM Player p
    RIGHT OUTER JOIN Salary s ON p.player_id = s.player_id;
""", conn2)

DatabaseError: Execution failed on sql '
    SELECT
        p.name_first || ' ' || p.name_last AS name_fl,
        p.birth_year || '-' || p.birth_month || '-' || p.birth_day AS birth_date,
        p.birth_city,
        p.weight,
        p.height,
        s.salary
    FROM Player p
    RIGHT OUTER JOIN Salary s ON p.player_id = s.player_id;
': RIGHT and FULL OUTER JOINs are not currently supported

EN EL CASO DE RIGHT OUTER JOIN SE ESPECÍFICA UQE OCURRE UN ERROR DEBIDO A QUE ACTUALMENTE NO ES SOPORTADO POR SQLite

##**Consulta 2**

In [ ]:
# Obtenga los nombres, fecha de nacimiento y cantidad de premios que ha obtenido cada jugador. Para esto utilice LEFT OUTER JOIN para obtener incluso a los jugadores que no han recibido algun premio
# Su código aquí...
pd.read_sql("""
    SELECT
        p.name_first || ' ' || p.name_last AS name_fl,
        p.birth_year || '-' || p.birth_month || '-' || p.birth_day AS birth_date,
        COUNT(a.award_id) AS total_awards
    FROM Player p
    LEFT OUTER JOIN player_award a ON p.player_id = a.player_id
    GROUP BY p.player_id;
""", conn2)

,name_fl,birth_date,total_awards
0,David Aardsma,1981-12-27,0
1,Hank Aaron,1934-2-5,16
2,Tommie Aaron,1939-8-5,0
3,Don Aase,1954-9-8,0
4,Andy Abad,1972-8-25,0
...,...,...,...
18841,Frank Zupo,1939-8-29,0
18842,Paul Zuvella,1958-10-31,0
18843,George Zuverink,1924-8-20,0
18844,Dutch Zwilling,1888-11-2,0


##**Consulta 3**

In [ ]:
# Obtenga la cantidad de jugadores que han pasado por una universidad, para esto utilice COUNT(DISTINCT \* ) donde \* corresponde a la columna a contar, para esto vea las tablas College y Player_College. ¿Por que es necesario ocupar DISTINCT? Genere la consulta sin agrupación y obtenga las columnas que ustedes estimen pertinentes
# Su código aquí...
pd.read_sql("""
    SELECT COUNT(DISTINCT pc.player_id) AS total_players
    FROM Player_College pc;
""", conn2)

,total_players
0,6575


##**Consulta 4**

In [ ]:
#veo la tabla team
pd.read_sql("PRAGMA table_info('appearances')",conn2)

,cid,name,type,notnull,dflt_value,pk
0,0,year,INTEGER,0,None,0
1,1,team_id,TEXT,0,None,0
2,2,league_id,TEXT,0,None,0
3,3,player_id,TEXT,0,None,0
4,4,g_all,NUMERIC,0,None,0
5,5,gs,NUMERIC,0,None,0
6,6,g_batting,INTEGER,0,None,0
7,7,g_defense,NUMERIC,0,None,0
8,8,g_p,INTEGER,0,None,0
9,9,g_c,INTEGER,0,None,0


LEFT OUTER JOIN

In [ ]:
#Proponga 3 consultas para distintas tablas con el fin de explorar. La restricción es que tiene que haber una cláusula LEFT OUTER JOIN, una de RIGHT OUTER JOIN, y una de FULL OUTER JOIN.
#listamos a todos los jugadores, aunque no tenga registro de partido
pd.read_sql("""
SELECT
    p.name_first || ' ' || p.name_last AS player_name,
    a.year,
    a.g_all AS total_games
FROM player p
LEFT OUTER JOIN appearances a ON p.player_id = a.player_id;
""", conn2)


,player_name,year,total_games
0,David Aardsma,2004.0,11
1,David Aardsma,2006.0,45
2,David Aardsma,2007.0,25
3,David Aardsma,2008.0,47
4,David Aardsma,2009.0,73
...,...,...,...
101132,Dutch Zwilling,1910.0,27
101133,Dutch Zwilling,1914.0,154
101134,Dutch Zwilling,1915.0,150
101135,Dutch Zwilling,1916.0,35


RIGHT OUTER JOIN  simulado

In [ ]:
# listamos todos los partidos asignados aunque no tengan jugadores
pd.read_sql("""
SELECT
    p.name_first || ' ' || p.name_last AS player_name,
    a.year,
    a.g_all AS total_games
FROM appearances a
LEFT OUTER JOIN player p ON a.player_id = p.player_id;
""", conn2)

,player_name,year,total_games
0,Ross Barnes,1871,31
1,Frank Barrows,1871,18
2,Dave Birdsall,1871,29
3,Fred Cone,1871,19
4,Charlie Gould,1871,31
...,...,...,...
100946,Barry Zito,2015,3
100947,Ben Zobrist,2015,59
100948,Ben Zobrist,2015,67
100949,Mike Zunino,2015,112


In [ ]:
# listamos todos los partidos y jugadores asignados aunque no tengan jugadores o partidos respectivamente
pd.read_sql("""
SELECT
    p.name_first || ' ' || p.name_last AS player_name,
    a.year,
    a.g_all AS total_games
FROM player p
LEFT OUTER JOIN appearances a ON p.player_id = a.player_id

UNION

SELECT
    p.name_first || ' ' || p.name_last AS player_name,
    a.year,
    a.g_all AS total_games
FROM appearances a
LEFT OUTER JOIN player p ON a.player_id = p.player_id;
""", conn2)

,player_name,year,total_games
0,None,2014.0,3
1,None,2014.0,9
2,None,2014.0,10
3,None,2014.0,22
4,None,2014.0,29
...,...,...,...
100852,Zoilo Versalles,1967.0,160
100853,Zoilo Versalles,1968.0,122
100854,Zoilo Versalles,1969.0,31
100855,Zoilo Versalles,1969.0,72


##**Fin Dinámica 2**

# <font color='red'>__LINK DE INTERÉS__: Vamos a reforzar los conocimientos sobre left join y right join</font>

El siguiente link de Microsoft refuerza y ejemplifica el uso de left join y right join, para visualizarlo, hacer click [aquí](https://learn.microsoft.com/en-us/office/client-developer/access/desktop-database-reference/left-join-right-join-operations-microsoft-access-sql).